<a href="https://colab.research.google.com/github/pea2/CSCE623-Project/blob/main/sentiment_v9_col_ww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code Source

https://www.kaggle.com/kritanjalijain/twitter-sentiment-analysis/notebook

https://www.kaggle.com/smitshah00/sentiment-analysis-ml-rnn

https://www.kaggle.com/lykin22/twitter-sentiment-analysis-with-naive-bayes-85-acc

Data Source

https://www.kaggle.com/kazanova/sentiment140



In [1]:
# print(sklearn.__version__)
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


Set current working directory to Google Drive 

In [2]:
# set current working directory to Google Drive to retrive data
from google.colab import drive 
drive.mount('/content/drive')
import os
%cd /content/drive/My\ Drive/Colab\ Notebooks/Thesis/
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Thesis


'/content/drive/My Drive/Colab Notebooks/Thesis'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Markdown as md
from IPython.display import display, Math, Latex

# Read in data

In [4]:
import pandas as pd
data_raw = pd.read_csv('02 1.6 mil sentiment(neg pos).csv', encoding = 'latin', header=None)

# Sequester test set

In [ ]:
data_raw.head()

In [5]:

from sklearn.model_selection import train_test_split

# first column is target column, separate it out for later train-test spliting
data_y = data_raw.loc[:,0] # first column
data_not_y = data_raw.iloc[:,1:6] # 2nd column to 6th column

# train-test split: 80-20
# set random state using numpy.random.RandomState so that each time calls the function, the split is not exactly the same

random_state = np.random.RandomState(9363)

X_train, X_test, y_train, y_test = train_test_split(data_not_y, data_y, 
                                                    test_size = 0.20, 
                                                    random_state=random_state, 
                                                    stratify = data_raw[0])

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape:  ', X_test.shape)
print('y_test.shape:  ', y_test.shape)
print('\ntraining set target counts per class:\n', y_train.value_counts(), '\n')
print('test set target counts per class:\n', y_test.value_counts())

X_train.shape:  (1280000, 5)
y_train.shape:  (1280000,)
X_test.shape:   (320000, 5)
y_test.shape:   (320000,)

training set target counts per class:
 4    640000
0    640000
Name: 0, dtype: int64 

test set target counts per class:
 4    160000
0    160000
Name: 0, dtype: int64


# Data exploration

### putting y back with rest of the columns and call the training set "data"

In [6]:
yy = pd.DataFrame(y_train)
data = yy.join(X_train)
# data.info()
# data.head()

### change column names to be more informative

In [7]:
data = data.rename(columns={0: 'target', 1: 'id', 2: 'date', 3: 'query', 4: 'username', 5: 'text'})
data.head()

# alternatively, change column name when reading in data
# data = pd.read_csv('02 1.6 mil sentiment(neg pos).csv',encoding='latin', names = ['target','id','date','query','username','text'])

,target,id,date,query,username,text
92685,0,1760170029,Sun May 10 20:50:18 PDT 2009,NO_QUERY,bigdog6286,but odds are we are not gonna make it ? http:...
414082,0,2060740674,Sat Jun 06 19:29:17 PDT 2009,NO_QUERY,TiffanyIPS,Is at Justins being forced to sing by about 3 ...
312468,0,2001536723,Tue Jun 02 01:12:40 PDT 2009,NO_QUERY,lorri79,back from fishing but no fish will try again ...
81414,0,1752713484,Sat May 09 22:44:34 PDT 2009,NO_QUERY,ricosuave22,hangin' out at home playing some online games....
397946,0,2056761369,Sat Jun 06 11:44:41 PDT 2009,NO_QUERY,lindatusti,I'm freakin' jealous of @irisvarsi She can go...


### checking if there are mising values

In [ ]:
missing_data = data.isna().sum().sort_values(ascending=False)
percentage_missing = round((data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)*100,2)
missing_info = pd.concat([missing_data,percentage_missing],keys=['Missing values','Percentage'],axis=1)
missing_info.style.background_gradient()

## Target values exploration

### negative positive classes

In [ ]:
# target classes: 0 and 4
data['target'].unique()

In [ ]:
# see class 0 text, observed that class 4 are negative tweets
pd.set_option('display.max_colwidth', None)
data[data['target']==0]['text'].head()

In [ ]:
# see class 4 text, since class 0 is negative tweets, it seems class 4 is positive tweets
pd.set_option('display.max_colwidth', None)
data[data['target']==4]['text'].head()

### replace class 0 with negative, class 4 with positive

In [8]:
# replace class 0 with negative, class 4 with positive
data['target'] = data['target'].replace([0,4], ['Negative', 'Positive'])
data['target'].unique()

array(['Negative', 'Positive'], dtype=object)

### ploting classes counts

In [ ]:
# plotting classes count in a bar chart
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
sns.countplot(data['target'], palette = ['orange', 'grey'])
plt.title('Bar chart for tweet classes',fontsize=16)
plt.show
data.target.value_counts()

In [ ]:
# plotting classes count in a pie chart
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5,5))
targets = data.groupby('target').size()
targets.plot(kind='pie', subplots=True, figsize=(5,5), autopct = "%.2f%%", colors=['grey','orange'])
plt.title("Pie chart for tweet classes",fontsize=16)
plt.ylabel("")
plt.legend()
plt.show()

In [ ]:
data['target'].value_counts()

## Text exploration

In [9]:
# compute length of each tweet, add the length column to dataframe
data['length'] = data.text.str.split().apply(len)

### tweet length

In [ ]:
# plot distribution of tweet length 

import seaborn as sns

fig = plt.figure(figsize=(10,5))

ax1 = fig.add_subplot(122)
sns.distplot(data[data['target']=='Positive']['length'], ax=ax1,color='orange')
describe = data.length[data.target=='Positive'].describe().to_frame().round(2)

ax2 = fig.add_subplot(121)
ax2.axis('off')
font_size = 20
bbox = [0, 0, 1, 1]
table = ax2.table(cellText = describe.values, rowLabels = describe.index, bbox=bbox, colLabels=describe.columns)
table.set_fontsize(font_size)
fig.suptitle('Distribution of possitive sentiment tweet length', fontsize=22)

plt.show()

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(10,5))

ax1 = fig.add_subplot(122)
sns.distplot(data[data['target']=='Negative']['length'], ax=ax1,color='grey')

ax2 = fig.add_subplot(121)
describe = data.length[data.target=='Negative'].describe().to_frame().round(2)
ax2.axis('off')
font_size = 20
bbox = [0, 0, 1, 1]
table = ax2.table(cellText = describe.values, rowLabels = describe.index, bbox=bbox, colLabels=describe.columns)
table.set_fontsize(font_size)
fig.suptitle('Distribution of negative sentiment tweet length', fontsize=22)

plt.show()

In [ ]:
# alternatively, plot word count distribution for both positive and negative 

p = data['length'][data.target == 'Positive']
n = data['length'][data.target == 'Negative']
plt.figure(figsize=(10,5))
plt.xlim(0,45)
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.hist([p, n], color=['orange','grey'], alpha=0.5, label=['positive','negative'], density=True)
plt.legend(loc='upper right')

### Top 10 users with most frequent tweets

In [ ]:
plt.figure(figsize=(10,5))
common_keyword=sns.barplot(x=data[data['target']=='Positive']['username'].value_counts()[:10].index, \
                           y=data[data['target']=='Positive']['username'].value_counts()[:10],palette='viridis')
common_keyword.set_xticklabels(common_keyword.get_xticklabels(),rotation=90)
common_keyword.set_ylabel('Positive tweet frequency',fontsize=18)
plt.title('Top 10 users with most frequent positive tweets',fontsize=22)
plt.show()

In [ ]:
# check out what does the most positive user tweet about, a bot!
pd.set_option('display.max_colwidth', None) 
data[data['username']=='what_bugs_u'].head()

In [ ]:
plt.figure(figsize=(10,5))
common_keyword=sns.barplot(x=data[data['target']=='Negative']['username'].value_counts()[:10].index, \
                           y=data[data['target']=='Negative']['username'].value_counts()[:10],palette='Spectral')
common_keyword.set_xticklabels(common_keyword.get_xticklabels(),rotation=90)
common_keyword.set_ylabel('Negative tweet frequency',fontsize=18)
plt.title('Top 10 users with most frequent negative tweets',fontsize=22)
plt.show()

In [ ]:
# check out what does the most nagative user tweet about, a bot! 
pd.set_option('display.max_colwidth', None)
data[data['username']=='lost_dog'].head()

### Wordclouds

In [ ]:
# 45-second run
from wordcloud import WordCloud, STOPWORDS

plt.figure(figsize=(10,7))
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1400, \
                       height=800, background_color="white").generate(" ".join(data[data.target=='Positive'].text))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.title('Most common words in positive sentiment tweets',fontsize=22)
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS

plt.figure(figsize=(10,7))
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1400, \
                       height=800, background_color="white").generate(" ".join(data[data.target=='Negative'].text))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.title('Most common words in negative sentiment tweets',fontsize=22)
plt.show()

# Text preprocessing

In [10]:
# remove noisy columns
print(data.columns)
data.drop(['id','date','query','username','length'], axis=1, inplace=True)
print(data.columns)

# replace positive sentiment with 1 and negative sentiment with 0
data.target = data.target.replace({'Positive' : 1, 'Negative' : 0})

Index(['target', 'id', 'date', 'query', 'username', 'text', 'length'], dtype='object')
Index(['target', 'text'], dtype='object')


### Emoji, smiley, contraction, cleaning preprocessing

### functions to clean text - Add new column to the data called "clean"
***----------------  RUN ---------------------------***

In [11]:
import re

################################################################################
################################################################################
def replace_emojis(t):
    '''
    This function replaces happy unicode emojis with "happy" and sad unicode emojis with "sad"
    '''
    emoji_happy = ["\U0001F600", "\U0001F601", "\U0001F602","\U0001F603","\U0001F604","\U0001F605", "\U0001F606", "\U0001F607", "\U0001F609", 
                  "\U0001F60A", "\U0001F642","\U0001F643","\U0001F923",r"\U0001F970","\U0001F60D", r"\U0001F929","\U0001F618","\U0001F617",
                  r"\U000263A", "\U0001F61A", "\U0001F619", r"\U0001F972", "\U0001F60B", "\U0001F61B", "\U0001F61C", r"\U0001F92A",
                  "\U0001F61D", "\U0001F911", "\U0001F917", r"\U0001F92D", r"\U0001F92B","\U0001F914","\U0001F910", r"\U0001F928", "\U0001F610", "\U0001F611",
                  "\U0001F636", "\U0001F60F","\U0001F612", "\U0001F644","\U0001F62C","\U0001F925","\U0001F60C","\U0001F614","\U0001F62A",
                  "\U0001F924","\U0001F634", "\U0001F920", r"\U0001F973", r"\U0001F978","\U0001F60E","\U0001F913", r"\U0001F9D0"]

    emoji_sad = ["\U0001F637","\U0001F912","\U0001F915","\U0001F922", r"\U0001F92E","\U0001F927", r"\U0001F975", r"\U0001F976", r"\U0001F974",
                  "\U0001F635", r"\U0001F92F", "\U0001F615","\U0001F61F","\U0001F641", r"\U0002639","\U0001F62E","\U0001F62F","\U0001F632",
                  "\U0001F633", r"\U0001F97A","\U0001F626","\U0001F627","\U0001F628","\U0001F630","\U0001F625","\U0001F622","\U0001F62D",
                  "\U0001F631","\U0001F616","\U0001F623"	,"\U0001F61E","\U0001F613","\U0001F629","\U0001F62B", r"\U0001F971",
                  "\U0001F624","\U0001F621","\U0001F620", r"\U0001F92C","\U0001F608","\U0001F47F","\U0001F480", r"\U0002620"]

    words = t.split()
    reformed = []
    for w in words:
        if w in emoji_happy:
            reformed.append("happy")
        elif w in emoji_sad:
            reformed.append("sad") 
        else:
            reformed.append(w)
    t = " ".join(reformed)
    return t
################################################################################
################################################################################
def replace_smileys(t):
    '''
    This function replaces happy smileys with "happy" and sad smileys with "sad"
    '''
    emoticons_happy = set([':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}', ':D',
      ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
      '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
      'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)', '<3'])

    emoticons_sad = set([':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
      ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
      ':c', ':{', '>:\\', ';('])  

    words = t.split()
    reformed = []
    for w in words:
        if w in emoticons_happy:
            reformed.append("happy")
        elif w in emoticons_sad:
            reformed.append("sad") 
        else:
            reformed.append(w)
    t = " ".join(reformed)
    return t
################################################################################
################################################################################
def replace_contractions(t):
    '''
    This function replaces english lanuage contractions, eg., "shouldn't" with "should not"
    '''
    cont = {"aren't" : 'are not', "can't" : 'cannot', "couln't": 'could not', "didn't": 'did not', "doesn't" : 'does not',
    "hadn't": 'had not', "haven't": 'have not', "he's" : 'he is', "she's" : 'she is', "he'll" : "he will", 
    "she'll" : 'she will',"he'd": "he would", "she'd":"she would", "here's" : "here is", 
    "i'm" : 'i am', "i've"	: "i have", "i'll" : "i will", "i'd" : "i would", "isn't": "is not", 
    "it's" : "it is", "it'll": "it will", "mustn't" : "must not", "shouldn't" : "should not", "that's" : "that is", 
    "there's" : "there is", "they're" : "they are", "they've" : "they have", "they'll" : "they will",
    "they'd" : "they would", "wasn't" : "was not", "we're": "we are", "we've":"we have", "we'll": "we will", 
    "we'd" : "we would", "weren't" : "were not", "what's" : "what is", "where's" : "where is", "who's": "who is",
    "who'll" :"who will", "won't":"will not", "wouldn't" : "would not", "you're": "you are", "you've":"you have",
    "you'll" : "you will", "you'd" : "you would", "mayn't" : "may not"}
    
    words = t.split()
    reformed = []
    for w in words:
        if w in cont:
          reformed.append(cont[w])
        else:
          reformed.append(w)
    t = " ".join(reformed)
    return t  
################################################################################
################################################################################
def remove_single_letter_words(t):
    '''
    This function removes words that are single characters
    '''
    words = t.split()
    reformed = []
    for w in words:
        if len(w) > 1:
            reformed.append(w)
    t = " ".join(reformed)
    return t 
################################################################################
################################################################################

# define emojis
emoji_pattern = re.compile("["
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def clean_data(t):
    '''
    This function cleans the tweets
    '''
    t = t.lower() # convert to lowercase
    t = replace_emojis(t) # replace unicode emojis with their associated emotions
    t = replace_smileys(t) # replace handwritten emojis with their associated emotions
    t = replace_contractions(t) # replace English short forms with their long form
    t = emoji_pattern.sub(r'', t) # remove emojis other than smiley emojis
    t = re.sub('\\\\u[0-9A-Fa-f]{4}','', t) # remove NON- ASCII characters
    t = re.sub("[0-9]", "", t) # remove numbers # re.sub("\d+", "", t)
    t = re.sub('#', '', t) # remove '#'
    t = re.sub('@[A-Za-z0–9]+', '', t) # remove '@'
    t = re.sub('@[^\s]+', '', t) # remove usernames
    t = re.sub('RT[\s]+', '', t) # remove retweet 'RT'
    t = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', t) # remove links (URLs/ links)
    t = re.sub('[!"$%&\'()*+,-./:@;<=>?[\\]^_`{|}~]', ' ', t)  ## replace punctuations with space ###
    t = t.replace('\\\\', '')
    t = t.replace('\\', '')
    t = remove_single_letter_words(t) # removes single letter words
    t = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(t).lower()).strip() # remove mentions and links in tweets
    
    return t
################################################################################
################################################################################

Add new column to the data called "clean"

In [12]:
print("Cleaning tweets...\n")
print("Convert to lowercase")
print("Replace unicode emojis with their associated emotions")
print("Replace handwritten emojis with their associated emotions")
print("Replace English short forms with their long form")
print("Remove all other unicoded emojis")
print("Remove NON- ASCII characters")
print("Remove numbers")
print("Remove \"#\" ")
print("Remove \"@\" ")
print("Remove usernames")
print("Remove \'RT\' ")
print("Remove URLs and links")
print("Replace punctuations with space")
print("Remove single letter words")
print("Remove mentions and links in tweets\n")

data['clean'] = data['text'].apply(clean_data)
print("Tweets have been cleaned.")

Cleaning tweets...

Convert to lowercase
Replace unicode emojis with their associated emotions
Replace handwritten emojis with their associated emotions
Replace English short forms with their long form
Remove all other unicoded emojis
Remove NON- ASCII characters
Remove numbers
Remove "#" 
Remove "@" 
Remove usernames
Remove 'RT' 
Remove URLs and links
Replace punctuations with space
Remove single letter words
Remove mentions and links in tweets

Tweets have been cleaned.


### check if data is clean and make WordClouds

In [ ]:
pd.set_option('display.max_colwidth', None)
data.clean.head()
# looks good

In [ ]:
# compare with prior cleaning the tweets
# see class 0 text, observed that class 4 are negative tweets
pd.set_option('display.max_colwidth', None)
data[data['target']==0]['text'].head()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1400, \
                       height=800, background_color="white").generate(" ".join(data[data.target==1].clean))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.title('Most common words in positive sentiment tweets (cleaned)',fontsize=22)
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1400, \
                       height=800, background_color="white").generate(" ".join(data[data.target==0].clean))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.title('Most common words in negative sentiment tweets (cleaned)',fontsize=22)
plt.show()

### Text normalization - Add new column to the data called "normal"
***---------------- 7 min RUN ---------------------------***

In [13]:
import nltk
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = nltk.corpus.stopwords.words('english')
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stopwords
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()
porter = nltk.PorterStemmer()

def normalize_data(doc):
  
  # tokenize text
  doc = nltk.tokenize.word_tokenize(doc)

  # remove stops words
  doc = [ w for w in doc if w not in (nltk_stopwords and sklearn_stopwords) ]

  # lemmatization
  doc = [wnl.lemmatize(w) for w in doc]

  # stemming
  doc = [porter.stem(w) for w in doc]

  # remove single letter
  doc = [w for w in doc if len(w)>1]

  return ' '.join(doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Add new column to the data called "normal"

In [14]:
# 6 minutes 43 seconds
data['normal'] = data['clean'].apply(normalize_data)

### compare with prior normalization

In [ ]:
pd.set_option('display.max_colwidth', None)
data.normal.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
data.clean.head()

### Convert text to word frequency vectors

In [ ]:
pd.set_option('display.max_colwidth', None)
data.head()

In [15]:
X = data['normal'].values
y = data['target'].values
print(X.shape)
print(y.shape)

(1280000,)
(1280000,)


## Pipeline

In [16]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = SGDClassifier(),
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

# https://stackoverflow.com/questions/50285973/pipeline-multiple-classifiers

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB


pipeline = Pipeline(steps=[
    ('tfidf_vec', TfidfVectorizer(sublinear_tf=True)),
    ('clf', ClfSwitcher())
], memory='/content/cache')

## Feature Reduction and Hyperparameter Optimization

In [35]:

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from time import time
import logging
from sklearn.metrics import confusion_matrix, classification_report

### Logreg

In [36]:
from sklearn.linear_model import LogisticRegression

params_tfidf = {
    'tfidf_vec__min_df': [0.0001, 0.00001, 0.000001, 1],
    'tfidf_vec__max_features': [None, 10000, 20000, 30000, 50000],
}

param_grid = [
    {   # log reg w/ elasticnet
        **params_tfidf,
        'clf__estimator': [LogisticRegression(max_iter=100000, solver='saga')],
        'clf__estimator__penalty': ['elasticnet'],
        'clf__estimator__l1_ratio': [0.1, 0.2, 0.8],
        'clf__estimator__C': [0.3, 0.7, 1.0, 10]
    },
    {   # log reg w/o elasticnet
        **params_tfidf,
        'clf__estimator': [LogisticRegression(max_iter=100000, solver='saga')],
        'clf__estimator__penalty': ['l1', 'l2'],
        'clf__estimator__C': [0.3, 0.7, 1.0, 10]
    },
    {
        **params_tfidf,
        'clf__estimator': [LinearSVC(max_iter=10000)],
        'clf__estimator__C': [0.3, 0.7, 1.0, 10]  
    }
]


In [37]:
search = HalvingGridSearchCV(pipeline, param_grid, scoring='f1', verbose=4, cv=5)

In [ ]:
# try 1:  min run
t0 = time()
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    search.fit(X, y)

print('done in %fs' % (time() - t0))


n_iterations: 5
n_required_iterations: 6
n_possible_iterations: 5
min_resources_: 5267
max_resources_: 1280000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 480
n_resources: 5267
Fitting 5 folds for each of 480 candidates, totalling 2400 fits
[CV 1/5] END clf__estimator=LogisticRegression(max_iter=100000, solver='saga'), clf__estimator__C=0.3, clf__estimator__l1_ratio=0.1, clf__estimator__penalty=elasticnet, tfidf_vec__max_features=None, tfidf_vec__min_df=0.0001;, score=(train=0.788, test=0.708) total time=   1.0s
[CV 2/5] END clf__estimator=LogisticRegression(max_iter=100000, solver='saga'), clf__estimator__C=0.3, clf__estimator__l1_ratio=0.1, clf__estimator__penalty=elasticnet, tfidf_vec__max_features=None, tfidf_vec__min_df=0.0001;, score=(train=0.802, test=0.730) total time=   0.9s
[CV 3/5] END clf__estimator=LogisticRegression(max_iter=100000, solver='saga'), clf__estimator__C=0.3, clf__estimator__l1_ratio=0.1, clf__estimator__penalty=elasticnet, tfidf_v

In [ ]:
# try 1 result
print("Best score: %0.3f" % search.best_score_)
print("Best parameters set:")
best_parameters = search.best_estimator_.get_params()
for param_name in sorted(param_logreg.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
# try 2: 11 min run
t0 = time()
search.fit(X, y)
print('done in %fs' % (time() - t0))


In [ ]:
# try 2 result
print("Best score: %0.3f" % search.best_score_)
print("Best parameters set:")
best_parameters = search.best_estimator_.get_params()
for param_name in sorted(param_logreg.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

### SVM

In [ ]:
# try 1
param_logreg = {
    'tfidf_vec__min_df': [0.0001, 0.00001, 0.000001],
    'tfidf_vec__max_features': [None, 10000, 15000, 20000],
    'tfidf_vec__sublinear_tf': [True, False],
    '__max_iter': [100000],

    '__C': [0.3, 0.7, 1.0]
}

In [ ]:
search = GridSearchCV(pipe_logreg, param_logreg, scoring='f1', cv=5)

## end of feature reduction

In [ ]:
# Convert a collection of raw documents to a matrix of TF-IDF features.
# 11-second run

from sklearn.feature_extraction.text import TfidfVectorizer

# preset some parameters in TfidfVectorizer
# max_docfreq = 0.8 # words show up at most 80% of the tweets - ignore high frequent words in all tweets
# min_docfreq = 0.2 # words show up at least 20% of the tweets - ignore rarely seen words in the corpus 
max_features = 20000 # set max feature number to be less than 1% of the training size of 1,280,000 observations
norm = 'l2' # default is l2 norm: The cosine similarity between two vectors is their dot product when l2 norm has been applied. 
# sublinear_tf=True calculate the tf as 1+log(tf)

vector = TfidfVectorizer(max_features=max_features, norm=norm, sublinear_tf=True)
X = vector.fit_transform(X)
print(f'Vector fitted.')
print('No. of feature_words: ', len(vector.get_feature_names()))

## Model building

### Validation Set
### Split training and validation set 75 - 25 split such that validation set size == test size = 320,000

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state=random_state)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print()
print("X_val", X_val.shape)
print("y_val", y_val.shape)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_valset(model, X_train, y_train, X_val, y_val):
    """ evaluate model and report validation set performance in confusion matrix and precision, recall, f1, support
    """
    # fit model
    model.fit(X_train, y_train)

    #accuracy of model on training data
    # acc_train=model.score(X_train, y_train)
    #accuracy of model on validation set
    acc_val = model.score(X_val, y_val)
    
    # print('Accuracy of model on training data : {}'.format(acc_train*100))
    print('Accuracy of model on validation set : {} \n'.format(acc_val*100))

    # Predict values for Test dataset
    y_pred = model.predict(X_val)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_val, y_pred))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_val, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Reds',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
# preset logreg parameters
max_iter = 100000
# penalty = 'elasticnet'
# l1_ratio = 0.5
solver = 'saga' # for larger dataset, faster, also supports elasticnet penalty in addition to ridge and lasso
logistic_regression = LogisticRegression(max_iter=max_iter, solver=solver)
evaluate_valset(logistic_regression, X_train, y_train, X_val, y_val)

### SVM Linear Kernel

In [ ]:
from sklearn.svm import LinearSVC
svm_linear = LinearSVC(random_state=random_state)
evaluate_valset(svm_linear, X_train, y_train, X_val, y_val)

### SVM Radial bias function (RBF) Kernel - 3hrs 45 min not done yet

In [ ]:
# from sklearn.svm import SVC
# svm_rbf = SVC(random_state=random_state)
# evaluate_valset(svm_rbf, X_train, y_train, X_val, y_val)

### Naive Bayes Bernoulli

In [ ]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
evaluate_valset(nb, X_train, y_train, X_val, y_val)

# original

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# def eval_val()
def evaluate(model):

    # fit model
    model.fit(X_train, y_train)

    #accuracy of model on training data
    # acc_train=model.score(X_train, y_train)
    #accuracy of model on test data
    acc_cv = model.score(X_cv, y_cv)
    
    # print('Accuracy of model on training data : {}'.format(acc_train*100))
    print('Accuracy of model on testing data : {} \n'.format(acc_cv*100))

    # Predict values for Test dataset
    y_pred = model.predict(X_cv)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_cv, y_pred))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_cv, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Reds',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()
evaluate(logistic_regression)

### SVM Linear Kernel

In [ ]:
from sklearn.svm import LinearSVC
svm_linear = LinearSVC()
evaluate(svm_linear)

### Naive Bayes Bernoulli

In [ ]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
evaluate(nb)